# **Optimization of Term Reduction in Typeless $\lambda$-Calculus**

## Import/Define required modules and functions

In [ ]:
class LambdaError(Exception):

    __errmsg = [
        "unrecognised error",
    ]

    def __init__(self, errDescription):
        if isinstance(errDescription, int):
            try:
                self._msg = LambdaError.__errmsg[errDescription]   
            except:
                self._msg = LambdaError.__errmsg[0]
        elif isinstance(errDescription, str):
            self._msg = errDescription
        else:
            self._msg = LambdaError.__errmsg[0]
        super().__init__(self._msg)

## Syntax

### Variables

The set $\mathbf{Var}$ is the set of variables that are atomic entities of the typeless $\lambda$-calculus, each of that refers to itself only.
We assume the existence of an infinite enumerable series of variables.

So, we propose the following computational model to represent a variable.<br/>
This model assumes that a variable is a decorated natural number.
A variable is represented as '`#k`' ($k\in\mathbb N$).

In [ ]:
class Var:

    __cvar = 0

    def __init__(self):
        self._data = Var.__cvar
        Var.__cvar += 1

    def __str__(self):
        return f"#{self._data}"

    def __eq__(self, another):
        if isinstance(another, Var):
            return self._data == another._data
        raise LambdaError("Var.__eq__ waits for an instance of Var"
                          f", but it received '{another}'")
        

Some examples of variables. 

In [ ]:
x, y, z = Var(), Var(), Var()

print(f"x = {x}\ny = {y}\nz = {z}")


x = #0
y = #1
z = #2


### Terms

The set $\mathbf{Term}$ of $\lambda$-terms (or briefly terms) is defined by the following rules.

---

$$\tag{$\Lambda$1}
\begin{equation}
\dfrac{x:\mathbf{Var}}{x:\mathbf{Term}}
\end{equation}
$$

$$
\tag{$\Lambda$2}
\begin{equation}
\dfrac{t_1:\mathbf{Term}\qquad t_2:\mathbf{Term}}{(t_1\ t_2):\mathbf{Term}}
\end{equation}
$$

$$\tag{$\Lambda$3}
\begin{equation}
\dfrac{x:\mathbf{Var}\qquad t:\mathbf{Term}}{(\operatorname{\lambda}x\mathop.t):\mathbf{Term}}
\end{equation}
$$

---

One usually uses the following rules for omitting parentheses

1. the outer parentheses omit always;
1. the term of the form $((t_1\ t_2)\ t_3)$ abbreviates to $t_1\ t_2\ t_3$;
1. the term of the form $(\operatorname{\lambda}x_1\mathop.(\operatorname{\lambda}x_2\mathop.t))$ abbreviates to $\operatorname{\lambda}x_1\mathop.\operatorname{\lambda}x_2\mathop.t$;
1. the term of the form $(\operatorname{\lambda}x\mathop.(t_1\ t_2))$ abbreviates to $\operatorname{\lambda}x\mathop.t_1\ t_2$.

The following classes represent $\lambda$-terms.<br/>
The class represents
* the atomic term $x$ where $x=\mathtt{\#}k$ ($ k\in\mathbb N$) as '$\mathtt{\$}k$'
* the application term $(t_1\ t_2)$ where $t_1$ and $t_2$ are terms as $\mathtt(t_1\ \mathtt.\ t_2\mathtt)$
* the abstraction term $\operatorname{\lambda}x\mathop.t$ where $x$ is a variable $\mathtt{\#}k$ and $t$ is a term as $\mathtt{(@}k\mathtt{ . }t\mathtt)$

In [ ]:
class Term:  # the basic abstract class for representing a term
    
    @property
    def kind(self):  # returns the kind of the term
        if isinstance(self, Atom):
            return "atom"
        if isinstance(self, Application):
            return "application"
        if isinstance(self, Abstraction):
            return "abstraction"

    def __str__(self):
        if self.kind == "atom":
            return f"${self._data}"
        if self.kind == "application":
            return f"({self._data[0]} . {self._data[1]})"
        else:  # self.kind == "absraction"
            return f"(@{self._data[0]}: {self._data[1]})"

    def __eq__(self, another):
        if isinstance(another, Term):
            if self.kind != another.kind:
                return False
            return self._data == another._data
        else:
            raise LambdaError(3)

    def callAsMethod(self, fun, *args):
        return fun(self, *args)


class Atom(Term):  # the class of terms created with the first rule

    def __init__(self, v):
        if isinstance(v, Var):
            self._data = v._data
        else:
            raise LambdaError("Atom.__init__ waits for an instance of Var"
                              f", but it received '{v}'")


class Application(Term):  # the class of terms created with the second rule

    def __init__(self, t1, t2):
        if isinstance(t1, Term) and isinstance(t2, Term):
            self._data = (t1, t2)
        else:
            raise LambdaError("Application.__init__ waits for two instances"
                              f" of Term, but it received '{t1}', '{t2}'")


class Abstraction(Term):  # the class of terms created with the third rule

    def __init__(self, v, t):
        if isinstance(v, Var) and isinstance(t, Term):
            self._data = (v._data, t)
        else:
            raise LambdaError("Abstraction.__init__ waits for an instance of"
                              " Var and an instance of Term"
                              f", but it receive '{v}' and '{t}'")

Some examples of terms

In [ ]:
tx, ty, tz = Atom(x), Atom(y), Atom(z)
tI = Abstraction(x, tx)
tK = Abstraction(x, Abstraction(y, tx))
tS = Abstraction(
         x,
         Abstraction(
             y,
             Abstraction(
                 z,
                 Application(
                     Application(tx, tz),
                     Application(ty, tz)))))

print(f"x = {tx}")
print(f"I = {tI}")
print(f"K = {tK}")
print(f"S = {tS}")

x = $0
I = (@0: $0)
K = (@0: (@1: $0))
S = (@0: (@1: (@2: (($0 . $2) . ($1 . $2)))))


### Paths

We use the concept of a ***path***.

A path is syntactically a string of '$\mathtt l$', '$\mathtt d$', and '$\mathtt r$'.
The set of paths is referred to as $\Pi$.

In [ ]:
def isPath(s):
    return isinstance(s, str) and len(s) == len([c for c in s if c in "ldr"])

A path is used for referring to a subterm of a term using the partially defined function $\operatorname{subref}:\mathbf{Term}\times\Pi\dashrightarrow\mathbf{Term}$.

$$\begin{array}{lll}
    \operatorname{subref}\ t\ \epsilon&=t&\textsf{for any term }t \\
    \operatorname{subref}\ (t_1\,t_2)\ \mathtt l\cdot\pi&=t_1&\textsf{for any terms }t_1,\ t_2\textsf{ and path }\pi \\
    \operatorname{subref}\ (t_1\,t_2)\ \mathtt r\cdot\pi&=t_2&\textsf{for any terms }t_1,\ t_2\textsf{ and path }\pi \\
    \operatorname{subref}\ (\lambda\,x\mathop{.}t)\ \mathtt d\cdot\pi&=t&\textsf{for any variable }x,\textsf{ term }t,\textsf{ and path }\pi \\
    \operatorname{subref}\ t\ \pi&\textsf{ is undefined }&\textsf{for all other cases}
\end{array}$$

The program realisation of this function is `subref(t: Term, p: Path) -> Term | None` specified here.<br/>
It returns the corresponding subterm or None if this subterm is undefined.

In [ ]:
def subref(t, p):
    if isinstance(t, Term) and isPath(p):
        if p == "":
            return t
        if p[0] == 'l' and t.kind == "application":
            return subref(t._data[0], p[1:])
        if p[0] == 'r' and t.kind == "application":
            return subref(t._data[1], p[1:])
        if p[0] == 'd' and t.kind == "abstraction":
            return subref(t._data[1], p[1:])
        # all other cases
        return None
    raise LambdaError("'subref' waits for an instance of Term and a path"
                      f", but it received '{t}' and '{p}'")

The set of paths for a term $t$ is defined as follows
$$\operatorname{\Pi}(t)=\{\pi\in\Pi\mid\operatorname{subref}\ t\ \pi\textsf{ is defined}\}.$$

In [ ]:
def paths(t):
    """collects all paths that refer to some correct subterm of 't'
    Result is a dictionary whose keys are paths determining
        the corresponding subterm
    """
    if isinstance(t, Term):
        result = {"": t}
        if t.kind == "atom":
            return result
        if t.kind == "application":
            return {**result,
                    **{("l" + key): val for (key, val) in
                       paths(subref(t, "l")).items()},
                    **{("r" + key): val for (key, val) in
                       paths(subref(t, "r")).items()}}
        # t.kind == "abstraction"
        return {**result,
                **{("d" + key): val for (key, val) in
                   paths(subref(t, "d")).items()}}
    raise LambdaError("'paths' waits for an instance of Term"
                      f", but it received '{t}'")

For example, the next cell computes the corresponding dictionary for combinator `tS`.

In [ ]:
pths = tS.callAsMethod(paths)
for key in pths.keys():
    print(f"'{key}': {pths[key]}")

'': (@0: (@1: (@2: (($0 . $2) . ($1 . $2)))))
'd': (@1: (@2: (($0 . $2) . ($1 . $2))))
'dd': (@2: (($0 . $2) . ($1 . $2)))
'ddd': (($0 . $2) . ($1 . $2))
'dddl': ($0 . $2)
'dddll': $0
'dddlr': $2
'dddr': ($1 . $2)
'dddrl': $1
'dddrr': $2


This example illustrates the following fact.

**Proposition.**
For any term $t$, $\Pi(t)$ is a prefix closed finite subset of $\Pi(t)$. 

In some sense, $\Pi(t)$ is the "skeleton" of $t$.
Terms with the same skeleton are similar.<br/>
This leads us to the function `similar`.

In [ ]:
def similar(t1, t2):
    if isinstance(t1, Term) and isinstance(t2, Term):
        return paths(t1).keys() == paths(t2).keys()
    raise LambdaError("'similar' waits for two instances of Term"
                      f", but it received '{t1}' and '{t2}'")

Let us consider the next examples.

In [ ]:
t1 = Application(tx, tI)
another_tI = Abstraction(y, ty)
t2 = Application(ty, another_tI)
print(f"{tI} and {another_tI}"
      f" are {''if similar(tI, another_tI) else 'not '}similar")
print(f"{t1} and {t2} are {''if similar(tI, another_tI) else 'not '}similar")

(@0: $0) and (@1: $1) are similar
($0 . (@0: $0)) and ($1 . (@1: $1)) are similar


Maximal paths in $\Pi(t)$ lead to variables.
There are two kinds of these paths:
* such ones that do not have a prefix, which refers to the abstraction-subterm with the variable equal to the variable, to which this path refers;
* and such ones that have a prefix, which refers to the abstraction-subterm with the variable equal to the variable, to which this path refers.

Paths of the first kind refer to free variables, and ones of the second kind refer to bound variables.

In [ ]:
def vars(t):
    """builds a dictionary, in which keys are refs to term variables,
    values are pairs constructed from the corresponding variable and
    the ref to the abstraction-superterm that bound the variable if
    it is bound or None elsewhen.
    """
    varoccs = {key: st._data
               for (key, st) in paths(t).items() if st.kind == "atom"}
    result = {}
    for key in varoccs:
        free = True
        for ie in range(1, len(key) + 1):
            subkey = key[: - ie]
            term = subref(t, subkey)
            if (term.kind == "abstraction" and
                term._data[0] == varoccs[key]):
                result[key] = (varoccs[key], subkey)
                free = False
                break
        if free:
            result[key] = (varoccs[key], None)
    return result

In [ ]:
print(tS)
for key in vars(tS):
    print(f"'{key}': {vars(tS)[key]}")

(@0: (@1: (@2: (($0 . $2) . ($1 . $2)))))
'dddll': (0, '')
'dddlr': (2, 'dd')
'dddrl': (1, 'd')
'dddrr': (2, 'dd')


In [ ]:
lst =[0, 1, 2, 3, 4]
for i in range(1, len(lst) + 1):
    print(lst[:- i])

[0, 1, 2, 3]
[0, 1, 2]
[0, 1]
[0]
[]
